# Image export (uncomment to run)

In [ ]:
# %%capture
# !pip install kaleido
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4


<IPython.core.display.Javascript object>

here's an example update. now let's change it.

# Code Preamble

In [ ]:
#@title
# FLAG refers to something I need to come back to

############ Points of reference fo counting: ############

# Yes/No questions can be normalized, because they must take one of those values (Yes, No, or blank). Normalizing ignores the blanks
# MCQs where you can pick 1 answer can be normalized to itself, e.g. oxytocin delivery method. 30% time was bolus, 70% of time was infusion. NOT total case numbers.

# Checkboxes can be normalized, but think about what the denominator should be
#### Indication = the denominator should be total # of all indications listed (e.g. of all the indications, obstructed labor made up 30% of them)
######## As opposed to denominator being total number of cases, where you might have obstructed labor making up 10% of the indications, if it was left blank many times

#### Theatre members present = the denominator should be the total # of cases (e.g. an anesthetist was present for 30% of cases)

#@title
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
df = pd.read_csv('https://raw.githubusercontent.com/lawtj/safe/master/safeeval.csv')
pd.options.plotting.backend = "plotly"


for row in range(0,100):              #iterate over rows 0-100, which are blank because the field didn't exist initially
  df.loc[row,'phase'] = "Pre-SAFE"    #replace cell with value

#first exclude 'never attended safe'
df = df[df['phase'] != "Never attended SAFE"]

#df2 = df.copy() # make a copy of the total dataframe
#df2 = df[df['phase'] == "Immediately post SAFE"] # set df2 to Immediately post safe only
#presafe = df[df['phase'] == "Pre-SAFE"] # set presafe df to pre-safe only
#df = df[df['phase'] == "Pre-SAFE"] # set df to pre-safe only
totalcases = len(df.index)

# label hospitals
df['hospital'] = df['anesthetist_id']
# somehow select anesthetist IDs that contain MB, etc. 
# See note in google keep about how to syntax for this
df = df.replace({'hospital':{
    'MB.*':'MB',
    'Z.*':'Z',
    'IR.*':'IR',
    'R.*':'R',
    'NJ.*':'NJ'
}}, regex=True)

# functions
# this returns n, % 
def crossfx(var1, var2):
    taba =  pd.crosstab(df[var1], df[var2], margins=True)
    tabb = pd.crosstab(df[var1], df[var2], normalize='columns', margins=True)
    tab2 = taba.join(tabb, lsuffix='_n', rsuffix = '_%')
    tab2.columns = tab2.columns.map(lambda x: tuple(x.split('_')))
    tab2 = (tab2.sort_index(ascending=[True, False] , axis=1)
                .rename_axis(columns=['Phase', 'count/pct'], axis=1)
        )
    tab2 = tab2[['Pre-SAFE', 'Immediately post SAFE', '6 months post SAFE', '12-mo post SAFE','All']]
    return taba, tab2

def crossfx2(var1, var2):
    taba =  pd.crosstab(df[var1], df[var2], margins=True)
    tabb = pd.crosstab(df[var1], df[var2], normalize='columns', margins=True)
    tab2 = taba.join(tabb, lsuffix='_n', rsuffix = '_%')
    return tab2

# creates barchart from tab a above
def barchart():
    global taba
    taba = taba[taba.index!= 'All']
    taba = taba[['All']]
    fig = go.Figure(go.Bar(x=taba.index, y=taba['All']))
    return fig

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Descriptives

## How many patients were observed per participant anesthetist?

### Count and crosstabulation

Cases observed per observer and participant

In [ ]:
#@title
t1 = pd.crosstab(df['observer_id'], df['anesthetist_id'], margins=False)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

anesthetist_id,IR-A,IR-B,IR-C,IR-E,IR-F,IR-G,IR-I,IR-J,MB-A,MB-B,...,Z-D,Z-E,Z-F,Z-G,Z-H,Z-I,Z-J,Z-K,Z-L,Z-N
observer_id,,,,,,,,,,,,,,,,,,,,,
A - Beti,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
B - Brendan,0,0,1,0,0,0,1,2,0,2,...,0,0,0,0,0,3,0,0,1,0
D - Dulla,0,3,0,0,0,0,5,0,0,0,...,3,0,12,7,2,0,0,6,4,6
M - Mutter,0,0,0,0,0,0,0,0,12,7,...,0,0,0,0,0,0,0,0,0,0
P - Privatus,4,6,0,1,4,2,7,2,12,3,...,19,6,9,1,7,3,3,25,9,4


In [ ]:
#@title
fig = px.imshow(t1, text_auto=True)
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cases per participant for each phase. Did any participant make up a large amount of any given phase?

In [ ]:
#@title
t1 = pd.crosstab(df['anesthetist_id'], df['phase'], normalize='columns')
t1 = t1.round(2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
fig = px.imshow(t1.T, text_auto=True)
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
t1 = pd.crosstab(df['anesthetist_id'], df['phase'])
fig = px.imshow(t1.T, text_auto=True)
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
fig = px.bar(df, x=df['anesthetist_id'].value_counts().index, y=df['anesthetist_id'].value_counts().values,
             labels={
                     'x': "Participant ID",
                     'y': "Count of cases",
                 },
             title='# of cases per participant')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#fig.write_image("fig1.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### number of those attended safe

How many observed had attended SAFE, vs never attended SAFE?

In [ ]:
t1 = pd.crosstab(df['anesthetist_id'], df['phase'])
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

phase,12-mo post SAFE,6 months post SAFE,Immediately post SAFE,Pre-SAFE
anesthetist_id,,,,
IR-A,0,0,1,3
IR-B,0,5,3,1
IR-C,0,0,0,1
IR-E,0,0,0,1
IR-F,0,0,0,4
IR-G,0,0,0,2
IR-I,6,4,3,0
IR-J,0,0,0,4
MB-A,0,4,6,15


In [ ]:
t1 = pd.crosstab(df['anesthetist_id'], df['phase']).reset_index()
print(' there are '+str(len(t1))+' unique IDs.')

t1['count'] = t1['12-mo post SAFE']+ t1['6 months post SAFE'] + t1['Immediately post SAFE']
print('there are ' + str(len(t1[t1['count']==0])) + ' people who were observed pre-safe, but were not observed in any post-safe phase')

print('there are ' + str(len(t1[((t1['count']>0) & (t1['Pre-SAFE']>0))])) + ' people who were observed both pre and post safe')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 there are 35 unique IDs.
there are 12 people who were observed pre-safe, but were not observed in any post-safe phase
there are 13 people who were observed both pre and post safe


Looks as though there are 35 unique anesthetist_ids in total.

12 people observed pre safe who didn't attend after.
Therefore, 13 people observed in the safe phases, and all of those were observed pre safe.

## Count of cases by hospital

In [ ]:
taba, t1 = crossfx('hospital', 'phase')
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Phase     Pre-SAFE       Immediately post SAFE           6 months post SAFE  \
count/pct        n     %                     n         %                  n   
hospital                                                                      
IR              16  0.16                     7  0.079545                  9   
MB              30  0.30                    19  0.215909                 10   
NJ               4  0.04                    11  0.125000                 10   
R               18  0.18                    12  0.136364                  1   
Z               32  0.32                    39  0.443182                 59   
All            100   NaN                    88       NaN                 89   

Phase               12-mo post SAFE            All            
count/pct         %               n         %    n         %  
hospital                                                      
IR         0.101124               6  0.139535   38  0.118750  
MB         0.112360              10  0.232558   69  0.215625  
NJ         0.112360               8  0.186047   33  0.103125  
R          0.011236               3  0.069767   34  0.106250  
Z          0.662921              16  0.372093  146  0.456250  
All             NaN              43       NaN  320       NaN

In [ ]:
barchart()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## What cadre were participants?

### Count

In [ ]:
taba, t1 = crossfx('cadre','phase')
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Phase                        Pre-SAFE           Immediately post SAFE  \
count/pct                           n         %                     n   
cadre                                                                   
Assistant nurse anaesthetist       15  0.151515                    14   
Nurse anesthetist                  84  0.848485                    75   
Other (specify)                     0  0.000000                     0   
All                                99       NaN                    89   

Phase                                  6 months post SAFE            \
count/pct                            %                  n         %   
cadre                                                                 
Assistant nurse anaesthetist  0.157303                  1  0.011364   
Nurse anesthetist             0.842697                 82  0.931818   
Other (specify)               0.000000                  5  0.056818   
All                                NaN                 88       NaN   

Phase                        12-mo post SAFE       All            
count/pct                                  n    %    n         %  
cadre                                                             
Assistant nurse anaesthetist               0  0.0   30  0.094044  
Nurse anesthetist                         43  1.0  284  0.890282  
Other (specify)                            0  0.0    5  0.015674  
All                                       43  NaN  319       NaN

### Figure

In [ ]:
barchart()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## ASA Category

In [ ]:
taba, t1 = crossfx('case_asa', 'phase')
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Phase          Pre-SAFE       Immediately post SAFE            \
count/pct             n     %                     n         %   
case_asa                                                        
ASA 1                 3  0.03                     5  0.056180   
ASA 2                46  0.46                    24  0.269663   
ASA 3                 0  0.00                     2  0.022472   
ASA 4                 1  0.01                     1  0.011236   
Not documented       50  0.50                    57  0.640449   
All                 100   NaN                    89       NaN   

Phase          6 months post SAFE           12-mo post SAFE            All  \
count/pct                       n         %               n         %    n   
case_asa                                                                     
ASA 1                           3  0.033708               1  0.023256   12   
ASA 2                          12  0.134831              20  0.465116  102   
ASA 3                           0  0.000000               0  0.000000    2   
ASA 4                           1  0.011236               3  0.069767    6   
Not documented                 73  0.820225              19  0.441860  199   
All                            89       NaN              43       NaN  321   

Phase                     
count/pct              %  
case_asa                  
ASA 1           0.037383  
ASA 2           0.317757  
ASA 3           0.006231  
ASA 4           0.018692  
Not documented  0.619938  
All                  NaN

In [ ]:
barchart()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Case Urgency

In [ ]:
taba, t1 = crossfx('case_urgency','phase')
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Phase        Pre-SAFE           Immediately post SAFE            \
count/pct           n         %                     n         %   
case_urgency                                                      
Elective           25  0.255102                    28  0.314607   
Emergency          73  0.744898                    61  0.685393   
All                98       NaN                    89       NaN   

Phase        6 months post SAFE           12-mo post SAFE            All  \
count/pct                     n         %               n         %    n   
case_urgency                                                               
Elective                     23  0.258427              16  0.372093   92   
Emergency                    66  0.741573              27  0.627907  227   
All                          89       NaN              43       NaN  319   

Phase                   
count/pct            %  
case_urgency            
Elective      0.288401  
Emergency     0.711599  
All                NaN

In [ ]:
t1.to_excel('urgency.xls')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-23-d129c86181b5>:1: FutureWarning:

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.



In [ ]:
barchart()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Initial Technique (spinal vs GA)

### Frequency

In [ ]:
#@title
t1 = df.case_initialtechnique.value_counts()
t2 = df.case_initialtechnique.value_counts()/totalcases
t3 = pd.DataFrame(pd.concat([t1,t2], axis=1))
t3.columns= ['count','percent of total cases']
t3

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,count,percent of total cases
Spinal,311,0.968847
General,4,0.012461


### Figure

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Indication

### Frequency of indication

In [ ]:
#@title
#######################
###############
########## you do NOT need to do this step. have a new method to create the 
########## crosstab tables first, and label after. See 'spinal' section below
########## just keeping it here so i don't have to type it out again
# replace 'checked' with type of indication in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'indication___1':{'Checked':'Obstructed labour'}})
df = df.replace({'indication___2':{'Checked':'previous scars'}})
df = df.replace({'indication___3':{'Checked':'Antepartum hemmorhage'}})
df = df.replace({'indication___4':{'Checked':'Placental abruption'}})
df = df.replace({'indication___5':{'Checked':'Placenta accreta/percreta'}})
df = df.replace({'indication___6':{'Checked':'Eclampsia'}})
df = df.replace({'indication___7':{'Checked':'Severe pre-eclampsia'}})
df = df.replace({'indication___8':{'Checked':'Fetal distress'}})
df = df.replace({'indication___9':{'Checked':'Cord prolapse'}})
df = df.replace({'indication___10':{'Checked':'Multiple pregnancy'}})
df = df.replace({'indication___11':{'Checked':'Breech presentation'}})
df = df.replace({'indication___12':{'Checked':'Other'}})

indicationlist = ['indication___1' , 'indication___2', 'indication___3',
                  'indication___4', 'indication___5', 'indication___6',
                  'indication___7','indication___8','indication___9',
                  'indication___10','indication___11','indication___12']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#indication N count
indicationN = pd.DataFrame
dfs = []

for i in indicationlist:
    t1 = pd.crosstab(df[i],df['phase'],margins=True)
    dfs.append(t1)

indicationN = pd.concat(dfs, axis=0, keys=indicationlist)
indicationN

#indicaton % count
indicationPct = pd.DataFrame
dfs = []

for i in indicationlist:
    t1 = pd.crosstab(df[i],df['phase'],margins=True, normalize='columns')
    dfs.append(t1)

indicationPct = pd.concat(dfs, axis=0, keys=indicationlist)

#Join two tables *together*
indication = pd.DataFrame()
indication = indicationN.join(indicationPct, lsuffix='_n', rsuffix='_%')
indication.columns = indication.columns.map(lambda x: tuple(x.split('_')))
indication = (indication.sort_index(ascending=[True, False] , axis=1)
            .rename_axis(columns=['phase', 'count_pct'], axis=1)
       )

##cleanup table
# set column order
indication = indication[['Pre-SAFE', 'Immediately post SAFE', '6 months post SAFE', '12-mo post SAFE','All']]

#remove multiindex
indication = indication.droplevel(0)
#remove unchecked and column %s
indication= indication.drop(['Unchecked', 'All'], axis=0)
indication = indication.sort_values([('All','%')], ascending=False)

indication
indicationsum = indication
indication

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

phase                Pre-SAFE       Immediately post SAFE            \
count_pct                   n     %                     n         %   
previous scars             51  0.51                    46  0.516854   
Obstructed labour          22  0.22                    16  0.179775   
Other                      16  0.16                    16  0.179775   
Fetal distress             11  0.11                     5  0.056180   
Breech presentation         0  0.00                     4  0.044944   
Multiple pregnancy          1  0.01                     5  0.056180   
Severe pre-eclampsia        0  0.00                     4  0.044944   
Placental abruption         1  0.01                     0  0.000000   
Eclampsia                   0  0.00                     1  0.011236   

phase                6 months post SAFE           12-mo post SAFE            \
count_pct                             n         %               n         %   
previous scars                       47  0.528090              23  0.534884   
Obstructed labour                    13  0.146067               6  0.139535   
Other                                14  0.157303               5  0.116279   
Fetal distress                       21  0.235955               8  0.186047   
Breech presentation                   3  0.033708               5  0.116279   
Multiple pregnancy                    4  0.044944               0  0.000000   
Severe pre-eclampsia                  1  0.011236               0  0.000000   
Placental abruption                   0  0.000000               0  0.000000   
Eclampsia                             0  0.000000               0  0.000000   

phase                 All            
count_pct               n         %  
previous scars        167  0.520249  
Obstructed labour      57  0.177570  
Other                  51  0.158879  
Fetal distress         45  0.140187  
Breech presentation    12  0.037383  
Multiple pregnancy     10  0.031153  
Severe pre-eclampsia    5  0.015576  
Placental abruption     1  0.003115  
Eclampsia               1  0.003115

### Figure

In [ ]:
#@title
#create a series of the value counts
indication = indication.value_counts()

#plot the value counts series
fig = go.Figure([go.Bar(x=indication.index, y=indication[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Other indications?

In [ ]:
#@title
df.indication_other.dropna()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1                                          Genital warts
6                                        Oligohydramnios
10                                              Big baby
11                                       Oligohydramnios
12                                                P-PROM
13                                              Big baby
15                        Pregnancy induced hypertension
16                                              Big Baby
23                                         Pre-eclampsia
26                                  Post-Dates / Overdue
34                                              Big baby
37                                              Big baby
42                                        Transverse lie
56                  PIH / Pregnancy induced hypertension
82                                              Big baby
98                    PIH Pregnancy induced hypertension
102                                             Big baby
105                            

## Pre-Eclampsia box

In [ ]:
#@title
# counts of yes and no for eclampsia actions
t1 = df.ecl_coag.value_counts()
t2 = df.ecl_antihtn.value_counts()
t3 = df.ecl_mgs04.value_counts()
t4 = df.ecl_avoidspinal.value_counts()
t5 = df.ecl_garsi.value_counts()

#glue eclampsia actions together and transpose
ecl = pd.concat([t1, t2, t3, t4, t5], axis=1, sort=False)
ecl = ecl.transpose()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Frequency of pre-E actions

In [ ]:
#@title
#rename variables
s = pd.Series(['Confirm platelet/coagulation values', 'Confirm anti-hypertensives available', 'Confirm MgSO4 available', 'Avoids spinal if platelets < 80,000', 'If eclampsia, proceeds to GA with RSI'])
ecl = ecl.set_index(s)
ecl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No,Yes
Confirm platelet/coagulation values,6.0,NaN
Confirm anti-hypertensives available,4.0,2.0
Confirm MgSO4 available,3.0,4.0
"Avoids spinal if platelets < 80,000",3.0,NaN
"If eclampsia, proceeds to GA with RSI",1.0,1.0


### Figure

In [ ]:
#@title
fig = go.Figure(data=[
  go.Bar(name = 'Yes', x=ecl.index, y=ecl['Yes']),
  go.Bar(name = 'No', x=ecl.index, y=ecl['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Type of Incision

### Count

In [ ]:
#@title
incision = df.incisiontype.value_counts()
incision

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Lower transverse    317
T-incision            1
Other                 1
Classical             1
Name: incisiontype, dtype: int64

### Figure

In [ ]:
#@title
fig = go.Figure([go.Bar(x=incision.index, y=incision[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Times: Informed, Incision and delivery time (Emergency surgeries only)

In [ ]:
#@title
df_emerg = df.copy()
df_emerg = df_emerg[df_emerg['case_urgency'] ==  "Emergency"]

#first concatenate the date and the times together
#df_emerg['datetime_informed'] = df_emerg['obs_date'].astype(str) + " " + df_emerg['time_informed']
#df_emerg['incision_datetime'] = df_emerg['obs_date'].astype(str) + " " + df_emerg.incisiontime
#df_emerg['datetime_birth'] = df_emerg.obs_date.astype(str) + " " + df_emerg.time_birth
#df_emerg['datetime_skinclosed'] = df_emerg.obs_date.astype(str) + " " + df_emerg.time_skinclosed

df_emerg['datetime_informed'] = df_emerg['time_informed']
df_emerg['incision_datetime'] = df_emerg['incisiontime']
df_emerg['datetime_birth'] = df['time_birth']
df_emerg['datetime_skinclosed'] = df_emerg['time_skinclosed']

#then convert the new datetimes to time format
df_emerg['datetime_informed'] = pd.to_datetime(df_emerg.datetime_informed)
df_emerg['incision_datetime'] = pd.to_datetime(df_emerg.incision_datetime)
df_emerg['datetime_birth'] = pd.to_datetime(df_emerg.datetime_birth)
df_emerg['datetime_skinclosed'] = pd.to_datetime(df_emerg.datetime_skinclosed)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#find difference between times
df_emerg['informedtoincision'] = df_emerg.incision_datetime - df_emerg.datetime_informed
df_emerg['timetodelivery'] = df_emerg.datetime_birth - df_emerg.incision_datetime
df_emerg['deliverytoclosure'] = df_emerg.datetime_skinclosed - df_emerg.datetime_birth
df_emerg['incisiontoclosure'] = df_emerg.datetime_skinclosed - df_emerg.incision_datetime

# convert times to seconds
df_emerg['informedtoincision'] = df_emerg['informedtoincision'].dt.seconds
df_emerg['timetodelivery'] = df_emerg['timetodelivery'].dt.seconds
df_emerg['deliverytoclosure'] = df_emerg['deliverytoclosure'].dt.seconds
df_emerg['incisiontoclosure'] = df_emerg['incisiontoclosure'].dt.seconds

#divide seconds into minutes
df_emerg['informedtoincision'] = df_emerg['informedtoincision']/60
df_emerg['incisiontoclosure'] = df_emerg['incisiontoclosure']/60
df_emerg['timetodelivery'] = df_emerg['timetodelivery']/60
df_emerg['deliverytoclosure'] = df_emerg['deliverytoclosure']/60

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
fig = go.Figure()
fig.add_trace(go.Box(y=df_emerg['informedtoincision'], name='anesthetist informed to incision', boxpoints='all'))
fig.add_trace(go.Box(y=df_emerg['timetodelivery'], name='incision to delivery to birth', boxpoints='all'))
fig.add_trace(go.Box(y=df_emerg['deliverytoclosure'], name='delivery to closure', boxpoints='all'))
fig.add_trace(go.Box(y=df_emerg['incisiontoclosure'], name='incision to closure', boxpoints='all'))

fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Baseline Vitals

In [ ]:
#@title
fig = go.Figure()
fig.add_trace(go.Box(y=df['baseline_hr'], name='HR', boxpoints='all'))
fig.add_trace(go.Box(y=df['baseline_sbp'], name = 'sbp', boxpoints='all'))
fig.add_trace(go.Box(y=df['baseline_dbp'], name = 'dbp', boxpoints='all'))
fig.add_trace(go.Box(y=df['baseline_spo2'], name = 'SpO2', boxpoints='all'))

fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### How much preeclampsia?


In [ ]:
#@title
pree = df.copy()
pree = pree[pree['baseline_sbp'] >=140]
t1 = pree['baseline_sbp'].count()
print('There are ',t1,'cases with SBP>140')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

There are  92 cases with SBP>140


### What proportion of time were baseline vitals measured?

Number of cases with recorded vital signs / all cases

In [ ]:
#@title
# start by counting number of nonblanks in each variable
# e.g number of nonblank basline HR
# df.baseline.hr.count() = ignores NaN by default

t1 = df.baseline_hr.count()/len(df.index) # percent of cases with baseline HR
t2 = df.baseline_sbp.count()/len(df.index) # percent of cases with baseline SBP
t3 = df.baseline_dbp.count()/len(df.index) # percent of cases with baseline DBP
t4 = df.baseline_spo2.count()/len(df.index) # percent of cases with baseline SpO2

baselinepct = {'% of cases measured': [t1, t2, t3, t4]}
baselinepct = pd.DataFrame(data=baselinepct)
baselinepct = baselinepct.set_index(pd.Index(['Heart rate', 'SBP', 'DBP', 'SpO2']))
baselinepct
#len(df.index) # number of rows in dataframe

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,% of cases measured
Heart rate,0.672897
SBP,0.978193
DBP,0.978193
SpO2,0.638629


## Team Members Present

In [ ]:
#@title
# replace 'checked' with which team members present in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'team___1':{'Checked':'Obstetrician resident'}})
df = df.replace({'team___2':{'Checked':'Obstetrician consultant'}})
df = df.replace({'team___3':{'Checked':'Medical officer'}})
df = df.replace({'team___4':{'Checked':'Intern'}})
df = df.replace({'team___5':{'Checked':'Anesthetist'}})
df = df.replace({'team___6':{'Checked':'Theatre nurse (runner)'}})
df = df.replace({'team___7':{'Checked':'Scrub nurse'}})
df = df.replace({'team___8':{'Checked':'Midwife'}})
df = df.replace({'team___9':{'Checked':'Neonatal term (doctor or nurse)'}})
df = df.replace({'team___10':{'Checked':'Medical student'}})
df = df.replace({'team___11':{'Checked':'Other'}})

# create dummy variables for team members
t1 = df.team___1
t2 = df.team___2
t3 = df.team___3
t4 = df.team___4
t5 = df.team___5
t6 = df.team___6
t7 = df.team___7
t8 = df.team___8
t9 = df.team___9
t10 = df.team___10
t11 = df.team___11

#join together dummies
team = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11], axis=0, sort=False)

#remove 'unchecked' values
team = team[team!='Unchecked']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Frequency of Team Members

FLAG need to normalize based on total number of cases

In [ ]:
#@title
t1 = team.value_counts()
t2 = team.value_counts()/totalcases
t3 = pd.DataFrame(pd.concat([t1,t2], axis=1))
t3.columns = ['count','% of all cases']
t3

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,count,% of all cases
Anesthetist,321,1.000000
Midwife,305,0.950156
Theatre nurse (runner),298,0.928349
Scrub nurse,251,0.781931
Intern,228,0.710280
Medical officer,159,0.495327
Other,119,0.370717
Medical student,97,0.302181
Obstetrician resident,70,0.218069
Obstetrician consultant,47,0.146417


### Figure

In [ ]:
#@title
#create a series of the value counts
team = team.value_counts()

#plot the value counts series
fig = go.Figure([go.Bar(x=team.index, y=team[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Which other team members?

In [ ]:
#@title
df.team_other.dropna()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2                 AMO
3                 AMO
4                 AMO
14     Another intern
28         2nd intern
            ...      
415               AMO
417               AMO
420               AMO
421               ANO
422               AMO
Name: team_other, Length: 119, dtype: object

# Equipment Check

## Monitoring equipment

In [ ]:
#@title
# replace 'checked' with names of things checked, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'equipment_monitoring___1':{'Checked':'BP Cuff'}})
df = df.replace({'equipment_monitoring___2':{'Checked':'Pulse oximeter'}})
df = df.replace({'equipment_monitoring___3':{'Checked':'ECG'}})
df = df.replace({'equipment_monitoring___4':{'Checked':'ETCO2'}})
df = df.replace({'equipment_airway___1':{'Checked':'Laryngoscope'}})
df = df.replace({'equipment_airway___2':{'Checked':'ETT'}})
df = df.replace({'equipment_airway___3':{'Checked':'LMA'}})
df = df.replace({'equipment_airway___4':{'Checked':'Bougie/stylet'}})
df = df.replace({'equipment_airway___5':{'Checked':'Facemask/breathing circuit'}})
df = df.replace({'equipment_airway___6':{'Checked':'Suction'}})
df = df.replace({'equipment_airway___7':{'Checked':'Ambu-bag'}})
df = df.replace({'equipment_drugs___1':{'Checked':'Oxygen'}})
df = df.replace({'equipment_drugs___2':{'Checked':'Ketamine'}})
df = df.replace({'equipment_drugs___3':{'Checked':'Volatile anesthetic'}})
df = df.replace({'equipment_drugs___4':{'Checked':'Pressor/adrenaline'}})
df = df.replace({'equipment_drugs___5':{'Checked':'Muscle relaxants'}})
df = df.replace({'equipment_drugs___6':{'Checked':'Antibiotics'}})
df = df.replace({'equipment_drugs___7':{'Checked':'Uterotonic'}})
df = df.replace({'equipment_others___1':{'Checked':'Anesthesia machine'}})
df = df.replace({'equipment_others___2':{'Checked':'Operating table with tilt'}})
df = df.replace({'equipment_others___3':{'Checked':'IV Fluids'}})
df = df.replace({'equipment_others___4':{'Checked':'Blood'}})

# create dummy variables for items checked
t1 = df.equipment_monitoring___1
t2 = df.equipment_monitoring___2
t3 = df.equipment_monitoring___3
t4 = df.equipment_monitoring___4
t5 = df.equipment_airway___1
t6 = df.equipment_airway___2
t7 = df.equipment_airway___3
t8 = df.equipment_airway___4
t9 = df.equipment_airway___5
t10 = df.equipment_airway___6
t11 = df.equipment_airway___7
t12 = df.equipment_drugs___1
t13 = df.equipment_drugs___2
t14 = df.equipment_drugs___3
t15 = df.equipment_drugs___4
t16 = df.equipment_drugs___5
t17 = df.equipment_drugs___6
t18 = df.equipment_drugs___7
t19 = df.equipment_others___1
t20 = df.equipment_others___2
t21 = df.equipment_others___3
t22 = df.equipment_others___4

#glue together the categories of checking items
monitoring = pd.concat([t1, t2, t3, t4], axis=0, sort=False)
airway = pd.concat([t5, t6, t7, t8, t9, t10, t11], axis=0, sort=False)
drugs = pd.concat([t12, t13, t14, t15, t16, t17, t18], axis=0, sort=False)
others = pd.concat([t19, t20, t21, t22], axis=0, sort=False)

#remove unchecked values
monitoring = monitoring[monitoring != 'Unchecked']
airway = airway[airway != 'Unchecked']
drugs = drugs[drugs != 'Unchecked']
others = others[others != 'Unchecked']

#Create frequency lists for each cateogry
monitoring = monitoring.value_counts()
airway = airway.value_counts()
drugs = drugs.value_counts()
others = others.value_counts()

#make dataframes out of the series, so we can add another column
monitoring = monitoring.to_frame() 
airway = airway.to_frame()
drugs = drugs.to_frame()
others = others.to_frame()

#create a column, with the category
monitoring['Check what'] = 'Monitoring equipment' 
airway['Check what'] = 'Airway equipment'
drugs['Check what'] = 'Drugs'
others['Check what'] = 'Others'

#glue together the dataframes, with categories included
checked = pd.concat([monitoring, airway, drugs, others], axis=0, sort=False)
checked = checked.reset_index() #make the index just numbers

checked.columns = ['Item', 'Count', 'Category'] #rename columns
checked['percent'] = checked.Count/totalcases
checked = checked[['Item', 'Count','percent','Category']]
checked
#checked['percent'] = checked['Count']/totalcases

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Item,Count,percent,Category
0,BP Cuff,320,0.996885,Monitoring equipment
1,Pulse oximeter,317,0.987539,Monitoring equipment
2,ECG,277,0.862928,Monitoring equipment
3,ETCO2,117,0.364486,Monitoring equipment
4,Facemask/breathing circuit,315,0.981308,Airway equipment
5,Bougie/stylet,308,0.959502,Airway equipment
6,Laryngoscope,304,0.947040,Airway equipment
7,ETT,301,0.937695,Airway equipment
8,LMA,293,0.912773,Airway equipment
9,Suction,284,0.884735,Airway equipment


In [ ]:
#@title
fig = go.Figure()
fig.add_trace(go.Bar(x=checked[checked['Category'] == 'Monitoring equipment']['Item'], y=checked[checked['Category'] == 'Monitoring equipment']['Count'], name='Monitoring equipment'))
fig.add_trace(go.Bar(x=checked[checked['Category'] == 'Airway equipment']['Item'], y=checked[checked['Category'] == 'Airway equipment']['Count'], name='Airway equipment'))
fig.add_trace(go.Bar(x=checked[checked['Category'] == 'Drugs']['Item'], y=checked[checked['Category'] == 'Drugs']['Count'], name='Drugs'))
fig.add_trace(go.Bar(x=checked[checked['Category'] == 'Others']['Item'], y=checked[checked['Category'] == 'Others']['Count'], name='Others'))

fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Spinal needle gauge


# Structured observation

## Effective communication

In [ ]:
commvars = ['comm_consent', 'comm_indication', 'checklist_signin', 'checklist_timeout', 'checklist_signout']
commnames = ['Consent', 'Indication', 'Sign-in', 'Time-out', 'Sign-out']

#calculate N
commN = pd.DataFrame()
dfs = []

for i in commvars:
    t1 = pd.crosstab(df[i],df['phase'], margins=True)
    dfs.append(t1)

commN = pd.concat(dfs, axis=0, keys=commnames)

#calculate percent

commPct = pd.DataFrame()
dfs = []

for i in commvars:
    t1 = pd.crosstab(df[i],df['phase'], margins=True, normalize = 'columns')
    dfs.append(t1)

commPct = pd.concat(dfs, axis=0, keys=commnames)

#join tables
#Join two tables *together*
communication = pd.DataFrame()
communication = commN.join(commPct, lsuffix='_n', rsuffix='_%')
communication.columns = communication.columns.map(lambda x: tuple(x.split('_')))
communication = (communication.sort_index(ascending=[True, False] , axis=1)
            .rename_axis(columns=['phase', 'count_pct'], axis=1)
       )

# set column order
communication = communication[['Pre-SAFE', 'Immediately post SAFE', '6 months post SAFE', '12-mo post SAFE','All']]

#drop no/all
communication = communication.drop(index=['No','All'], level=1)

communication

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

phase                   Pre-SAFE           Immediately post SAFE            \
count_pct                      n         %                     n         %   
           comm_consent                                                      
Consent    Yes                93  0.939394                    89  1.000000   
Indication Yes                68  0.693878                    75  0.862069   
Sign-in    Yes                12  0.120000                    46  0.516854   
Time-out   Yes                12  0.120000                    45  0.505618   
Sign-out   Yes                 3  0.030000                    18  0.202247   

phase                   6 months post SAFE           12-mo post SAFE  \
count_pct                                n         %               n   
           comm_consent                                                
Consent    Yes                          89  1.000000              43   
Indication Yes                          88  0.988764              43   
Sign-in    Yes                          51  0.573034              28   
Time-out   Yes                          47  0.534091              19   
Sign-out   Yes                          26  0.295455              13   

phase                              All            
count_pct                       %    n         %  
           comm_consent                           
Consent    Yes           1.000000  314  0.981250  
Indication Yes           1.000000  274  0.864353  
Sign-in    Yes           0.651163  137  0.426791  
Time-out   Yes           0.441860  123  0.384375  
Sign-out   Yes           0.302326   60  0.187500

### Consent

In [ ]:
#@title
t1 = df.comm_consent.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Yes    0.98125
No     0.01875
Name: comm_consent, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Discusses indication

In [ ]:
#@title
t1 = df.comm_indication.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Yes    0.864353
No     0.135647
Name: comm_indication, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## WHO Checklist

In [ ]:
#@title
t1 = df.checklist_signin.value_counts()/totalcases
t2 = df.checklist_timeout.value_counts()/totalcases
t3 = df.checklist_signout.value_counts()/totalcases

checklist = pd.concat([t1, t2, t3], axis=1, sort=False)
checklist.columns = ['Sign in', 'Time out', 'Sign out']
checklist=checklist.transpose()
checklist

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No,Yes
Sign in,0.573209,0.426791
Time out,0.613707,0.383178
Sign out,0.809969,0.186916


In [ ]:
#@title
fig = go.Figure(data=[
  go.Bar(name = 'Yes', x=checklist.index, y=checklist['Yes']),
  go.Bar(name = 'No', x=checklist.index, y=checklist['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Preparation

### Percent of answers

In [ ]:
preplist = ['prep_preop', 'prep_hb', 'prep_machine', 'prep_gadrugs',
'prep_airwaychecked', 'prep_suction' ,'prep_vasopressor',
'prep_neonatal' ,'prep_ivaccess','prep_fluids']

prepnames = ['Pre-operative anesthetic assessment', 'Checks recent Hb level', 'Anesthetic machine checked', 'Availability of GA drugs checked', 
             'Airway equipment available checked', 'Suction present & working', 
             'Vasopressor present', 'Neonatal resuscitation equipment available', 'Obtains IV access',
             'Attaches running fluids']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#calculate N
prepN = pd.DataFrame()
dfs = []

for i in preplist:
    t1 = pd.crosstab(df[i],df['phase'], margins=True)
    dfs.append(t1)

prepN = pd.concat(dfs, axis=0, keys=prepnames)

#calculate percent

prepPct = pd.DataFrame()
dfs = []

for i in preplist:
    t1 = pd.crosstab(df[i],df['phase'], margins=True, normalize = 'columns')
    dfs.append(t1)

prepPct = pd.concat(dfs, axis=0, keys=prepnames)

#join tables
#Join two tables *together*
prep = pd.DataFrame()
prep = prepN.join(prepPct, lsuffix='_n', rsuffix='_%')
prep.columns = prep.columns.map(lambda x: tuple(x.split('_')))
prep = (prep.sort_index(ascending=[True, False] , axis=1)
            .rename_axis(columns=['phase', 'count_pct'], axis=1)
       )

# set column order
prep = prep[['Pre-SAFE', 'Immediately post SAFE', '6 months post SAFE', '12-mo post SAFE','All']]

#drop no/all
prep = prep.drop(index=['No','All'], level=1)

prep

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

phase                                          Pre-SAFE            \
count_pct                                             n         %   
Pre-operative anesthetic assessment        Yes       32  0.320000   
Checks recent Hb level                     Yes       42  0.428571   
Anesthetic machine checked                 Yes       44  0.440000   
Availability of GA drugs checked           Yes       64  0.640000   
Airway equipment available checked         Yes       86  0.860000   
Suction present & working                  Yes       73  0.730000   
Vasopressor present                        Yes      100  1.000000   
Neonatal resuscitation equipment available Yes       92  0.920000   
Obtains IV access                          Yes       99  1.000000   
Attaches running fluids                    Yes       99  1.000000   

phase                                          Immediately post SAFE  \
count_pct                                                          n   
Pre-operative anesthetic assessment        Yes                    45   
Checks recent Hb level                     Yes                    35   
Anesthetic machine checked                 Yes                    55   
Availability of GA drugs checked           Yes                    57   
Airway equipment available checked         Yes                    88   
Suction present & working                  Yes                    82   
Vasopressor present                        Yes                    89   
Neonatal resuscitation equipment available Yes                    88   
Obtains IV access                          Yes                    88   
Attaches running fluids                    Yes                    87   

phase                                                    6 months post SAFE  \
count_pct                                              %                  n   
Pre-operative anesthetic assessment        Yes  0.505618                 61   
Checks recent Hb level                     Yes  0.397727                 37   
Anesthetic machine checked                 Yes  0.617978                 79   
Availability of GA drugs checked           Yes  0.640449                 72   
Airway equipment available checked         Yes  0.988764                 81   
Suction present & working                  Yes  0.921348                 80   
Vasopressor present                        Yes  1.000000                 85   
Neonatal resuscitation equipment available Yes  0.988764                 88   
Obtains IV access                          Yes  1.000000                 89   
Attaches running fluids                    Yes  1.000000                 89   

phase                                                    12-mo post SAFE  \
count_pct                                              %               n   
Pre-operative anesthetic assessment        Yes  0.685393              38   
Checks recent Hb level                     Yes  0.435294              31   
Anesthetic machine checked                 Yes  0.908046              40   
Availability of GA drugs checked           Yes  0.808989              43   
Airway equipment available checked         Yes  0.910112              40   
Suction present & working                  Yes  0.898876              37   
Vasopressor present                        Yes  0.955056              43   
Neonatal resuscitation equipment available Yes  0.988764              42   
Obtains IV access                          Yes  1.000000              42   
Attaches running fluids                    Yes  1.000000              41   

phase                                                     All            
count_pct                                              %    n         %  
Pre-operative anesthetic assessment        Yes  0.883721  176  0.548287  
Checks recent Hb level                     Yes  0.738095  145  0.463259  
Anesthetic machine checked                 Yes  0.930233  218  0.683386  
Availability of GA drugs checked           Yes  1.000000  236  0.735202  


In [ ]:
#prep.to_excel('prepdescriptives.xls')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### What preop assessment was done?

In [ ]:
#@title
# replace 'checked' with which anesthetic assessments were done in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'prep_preop_parts___1':{'Checked':'Airway exam'}})
df = df.replace({'prep_preop_parts___2':{'Checked':'CVS/Resp exam'}})
df = df.replace({'prep_preop_parts___3':{'Checked':'Co-morbidities'}})
df = df.replace({'prep_preop_parts___4':{'Checked':'Allergies'}})

# create dummy variables for team members
t1 = df.prep_preop_parts___1
t2 = df.prep_preop_parts___2
t3 = df.prep_preop_parts___3
t4 = df.prep_preop_parts___4

#join together dummies
prep_preop = pd.concat([t1, t2, t3, t4], axis=0, sort=False)

#remove 'unchecked' values
prep_preop = prep_preop[prep_preop!='Unchecked']
prep_preop.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Airway exam       171
Allergies         168
CVS/Resp exam     118
Co-morbidities    103
dtype: int64

In [ ]:
prep_preop.value_counts(normalize=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Airway exam       0.305357
Allergies         0.300000
CVS/Resp exam     0.210714
Co-morbidities    0.183929
dtype: float64

### What machine checks were done?

In [ ]:
#@title
# replace 'checked' with which parts of the machine were checked, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'prep_machine_parts___1':{'Checked':'Adequate O2 supply'}})
df = df.replace({'prep_machine_parts___2':{'Checked':'Working breathing circuit (pressure/leak test)'}})

# create dummy variables for team members
t1 = df.prep_machine_parts___1
t2 = df.prep_machine_parts___2

#join together dummies
prep_machine = pd.concat([t1, t2], axis=0, sort=False)

#remove 'unchecked' values
prep_machine = prep_machine[prep_machine!='Unchecked']
prep_machine.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Adequate O2 supply                                215
Working breathing circuit (pressure/leak test)    213
dtype: int64

### What airway equipment was checked

In [ ]:
#@title
# replace 'checked' with which airway checkes were done in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'prep_airwaychecked_which___1':{'Checked':'Laryngoscope with light'}})
df = df.replace({'prep_airwaychecked_which___2':{'Checked':'ETT with cuff'}})

# create dummy variables for team members
t1 = df.prep_airwaychecked_which___1
t2 = df.prep_airwaychecked_which___2

#join together dummies
prep_airway = pd.concat([t1, t2], axis=0, sort=False)

#remove 'unchecked' values
prep_airway = prep_airway[prep_airway!='Unchecked']
prep_airway.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ETT with cuff              290
Laryngoscope with light    289
dtype: int64

### What vasopressors were available?

In [ ]:
#@title
# replace 'checked' with which vasopressors were available, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'prep_vasopressor_parts___1':{'Checked':'Adrenaline'}})
df = df.replace({'prep_vasopressor_parts___2':{'Checked':'Ephedrine'}})
df = df.replace({'prep_vasopressor_parts___3':{'Checked':'Phenylephrine'}})
df = df.replace({'prep_vasopressor_parts___4':{'Checked':'Metaraminol'}})

# create dummy variables for team members
t1 = df.prep_vasopressor_parts___1
t2 = df.prep_vasopressor_parts___2
t3 = df.prep_vasopressor_parts___3
t4 = df.prep_vasopressor_parts___4

#join together dummies
prep_vasopressor = pd.concat([t1, t2, t3, t4], axis=0, sort=False)

#remove 'unchecked' values
prep_vasopressor = prep_vasopressor[prep_vasopressor!='Unchecked']
prep_vasopressor.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Adrenaline       293
Ephedrine        255
Phenylephrine      1
dtype: int64

### What neonatal equipment was available?

In [ ]:
#@title
# replace 'checked' with which neonatal equipment was available, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'prep_neonatal_which___1':{'Checked':'Ambu-bag'}})
df = df.replace({'prep_neonatal_which___2':{'Checked':'Mask'}})
df = df.replace({'prep_neonatal_which___3':{'Checked':'Towel/blanket'}})
df = df.replace({'prep_neonatal_which___4':{'Checked':'Warmer'}})
df = df.replace({'prep_neonatal_which___5':{'Checked':'Oxygen'}})

# create dummy variables for team members
t1 = df.prep_neonatal_which___1
t2 = df.prep_neonatal_which___2
t3 = df.prep_neonatal_which___3
t4 = df.prep_neonatal_which___4
t5 = df.prep_neonatal_which___5

#join together dummies
prep_neonatal = pd.concat([t1, t2, t3, t4, t5], axis=0, sort=False)

#remove 'unchecked' values
prep_neonatal = prep_neonatal[prep_neonatal!='Unchecked']
prep_neonatal.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ambu-bag         298
Towel/blanket    298
Mask             292
Warmer           253
Oxygen           188
dtype: int64

# Outcomes

## Apgars

In [ ]:
#@title
fig = go.Figure()
fig.add_trace(go.Box(y=df['apgar1'], name='1-minute Apgar', boxpoints='all'))
fig.add_trace(go.Box(y=df['apgar5'], name = '5-minute Apgar', boxpoints='all'))

fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Hospital outcome

In [ ]:
#@title
t1 = df.final_ordeath.value_counts()
t2 = df.final_hospitaldeath.value_counts()
t3 = df.final_pttransfer.value_counts()

final = pd.concat([t1, t2, t3], axis=1, sort=False)
final.columns = ['OR death', 'Hospital death', 'Patient transfer']
final=final.transpose()
final

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No
OR death,320
Hospital death,258
Patient transfer,257


In [ ]:
#@title
fig = go.Figure(data=[
  #go.Bar(name = 'Yes', x=final.index, y=final['Yes']),
  go.Bar(name = 'No', x=final.index, y=final['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



---


# Spinal Packet

What spinal drug was used?

In [ ]:
#@title
df.spinal_drug.value_counts(normalize=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bupivicaine - 0.5%     242
Lidocaine - 5%          20
Bupivicaine - 0.25%     12
Name: spinal_drug, dtype: int64

What dose was in the spinal? ***FLAG*** Need to combine with drug type

## Dosages of bupivicaine 0.5%


In [ ]:
#@title
df.spinal_dose.dropna()
bupidose = df['spinal_dose'].loc[df['spinal_drug'] == 'Bupivicaine - 0.5%']

#@title
fig = go.Figure(data=[
  go.Histogram(x=bupidose),
])

fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

What adjuncts were used?


In [ ]:
#@title
# replace 'checked' with which adjuncts were used in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'spinal_adjunct___1':{'Checked':'Opioid'}})
df = df.replace({'spinal_adjunct___2':{'Checked':'Dextrose'}})

# create dummy variables for indication
t1 = df.spinal_adjunct___1
t2 = df.spinal_adjunct___2

#join together dummies
adjunct = pd.concat([t1, t2], axis=0, sort=False)

#remove 'unchecked' values
adjunct = adjunct[adjunct!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
adjunct.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Dextrose    21
dtype: int64

## Spinal behaviour

In [ ]:
#list of spinal variables
spinalvars = ['spinal_hat','spinal_mask','spinal_gloves','spinal_clean',
               'spinal_sterile', 'spinal_wedge', 'spinal_vitals','spinal_height',
               'spinal_vasopressor','spinal_abx','spinal_oxytocin','spinal_oxytocintime',
               'spinal_present']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Crosstab all spinal variables with count

In [ ]:
# Crosstab all spinal variables with count
spinaldata = pd.DataFrame()
dfs = []

for i in spinalvars:
    t1 = pd.crosstab(df[i], df['phase'], margins=True)
    dfs.append(t1)

spinaldata = pd.concat(dfs, axis=0, keys=spinalvars)

# Crosstab all variables with percent
spinaldatapct = pd.DataFrame()
dfs = []

for i in spinalvars:
    t1 = pd.crosstab(df[i], df['phase'], margins=True, normalize='columns')
    dfs.append(t1)

spinaldatapct = pd.concat(dfs, axis=0, keys=spinalvars)

#Join two tables *together*
spinalfinaldata = pd.DataFrame()
spinalfinaldata = spinaldata.join(spinaldatapct, lsuffix='_n', rsuffix='_%')
spinalfinaldata.columns = spinalfinaldata.columns.map(lambda x: tuple(x.split('_')))
spinalfinaldata = (spinalfinaldata.sort_index(ascending=[True, False] , axis=1)
            .rename_axis(columns=['phase', 'count_pct'], axis=1)
       )

# set column order
spinalfinaldata = spinalfinaldata[['Pre-SAFE', 'Immediately post SAFE', '6 months post SAFE', '12-mo post SAFE','All']]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
spinalfinaldata = spinalfinaldata.round(2).fillna('1.0')
spinalfinaldata = spinalfinaldata.drop(index=['No','All'], level=1)
spinalfinaldata.sort_values([('All','%')], ascending=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

phase                   Pre-SAFE       Immediately post SAFE        \
count_pct                      n     %                     n     %   
spinal_gloves       Yes       95   1.0                    84   1.0   
spinal_hat          Yes       95   1.0                    84   1.0   
spinal_mask         Yes       95   1.0                    84   1.0   
spinal_clean        Yes       92  0.98                    83   1.0   
spinal_vitals       Yes       93  0.98                    84   1.0   
spinal_oxytocin     Yes       93  0.98                    84   1.0   
spinal_sterile      Yes       63  0.67                    73  0.87   
spinal_abx          Yes       63  0.66                    74  0.88   
spinal_present      Yes       74   0.8                    66   0.8   
spinal_wedge        Yes       41  0.43                    57  0.69   
spinal_vasopressor  Yes       43  0.49                    37  0.67   
spinal_oxytocintime Yes       51  0.54                    48  0.58   
spinal_height       Yes       30  0.32                    55  0.65   

phase                   6 months post SAFE       12-mo post SAFE        All  \
count_pct                                n     %               n     %    n   
spinal_gloves       Yes                 89   1.0              41   1.0  309   
spinal_hat          Yes                 87  0.98              41   1.0  307   
spinal_mask         Yes                 86  0.97              40  0.98  305   
spinal_clean        Yes                 89   1.0              41   1.0  305   
spinal_vitals       Yes                 89   1.0              40  0.98  306   
spinal_oxytocin     Yes                 89   1.0              39  0.95  305   
spinal_sterile      Yes                 84  0.94              41   1.0  261   
spinal_abx          Yes                 81  0.93              38  0.95  256   
spinal_present      Yes                 66  0.76              36   0.9  242   
spinal_wedge        Yes                 76  0.86              33   0.8  207   
spinal_vasopressor  Yes                 27  0.71              14   0.7  121   
spinal_oxytocintime Yes                 52   0.6              26  0.63  177   
spinal_height       Yes                 63  0.71              29  0.71  177   

phase                          
count_pct                   %  
spinal_gloves       Yes   1.0  
spinal_hat          Yes  0.99  
spinal_mask         Yes  0.99  
spinal_clean        Yes  0.99  
spinal_vitals       Yes  0.99  
spinal_oxytocin     Yes  0.99  
spinal_sterile      Yes  0.85  
spinal_abx          Yes  0.84  
spinal_present      Yes   0.8  
spinal_wedge        Yes  0.67  
spinal_vasopressor  Yes   0.6  
spinal_oxytocintime Yes  0.58  
spinal_height       Yes  0.57

In [ ]:
#spinalfinaldata.to_excel('spinaldescriptives.xls')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
t1 = spinaldatapct.drop(index=['No'], level=1)
t1.reset_index()
#t1.drop(['phase'], axis=1)
#df.drop(['Q', 'R'], axis=1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

phase,level_0,level_1,12-mo post SAFE,6 months post SAFE,Immediately post SAFE,Pre-SAFE,All
0,spinal_hat,Yes,1.000000,0.977528,1.000000,1.000000,0.993528
1,spinal_mask,Yes,0.975610,0.966292,1.000000,1.000000,0.987055
2,spinal_gloves,Yes,1.000000,1.000000,1.000000,1.000000,1.000000
3,spinal_clean,Yes,1.000000,1.000000,1.000000,0.978723,0.993485
4,spinal_sterile,Yes,1.000000,0.943820,0.869048,0.670213,0.847403
5,spinal_wedge,Yes,0.804878,0.863636,0.686747,0.431579,0.674267
6,spinal_vitals,Yes,0.975610,1.000000,1.000000,0.978947,0.990291
7,spinal_height,Yes,0.707317,0.707865,0.654762,0.315789,0.572816
8,spinal_vasopressor,Yes,0.700000,0.710526,0.672727,0.494253,0.605000
9,spinal_abx,Yes,0.950000,0.931034,0.880952,0.663158,0.836601


### What vital signs were monitored?

In [ ]:
#@title
# replace 'checked' with which vital signs were monitored were used in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'spinal_vitals_which___1':{'Checked':'BP every 5 min'}})
df = df.replace({'spinal_vitals_which___2':{'Checked':'HR every 5 min'}})
df = df.replace({'spinal_vitals_which___3':{'Checked':'Continuous SpO2'}})

# create dummy variables for indication
t1 = df.spinal_vitals_which___1
t2 = df.spinal_vitals_which___2
t3 = df.spinal_vitals_which___3

#join together dummies
spinal_vitals = pd.concat([t1, t2, t3], axis=0, sort=False)

#remove 'unchecked' values
spinal_vitals = spinal_vitals[spinal_vitals!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

***FLAG*** = need to normalize the following, where the denominator is # of cases

In [ ]:
#@title
t1 = spinal_vitals.value_counts()
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BP every 5 min     303
HR every 5 min     302
Continuous SpO2    298
dtype: int64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Oxytocin dose

In [ ]:
#@title
t1 = df.spinal_oxytocin_dose.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

10 units     0.756494
>10 units    0.240260
5 units      0.003247
Name: spinal_oxytocin_dose, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Oxytocin method

In [ ]:
#@title
t1 = df.spinal_oxytocin_how.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bolus       0.678571
Both        0.256494
Infusion    0.064935
Name: spinal_oxytocin_how, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Adverse events

In [ ]:
#@title
t1 = df.adverse_failed_spinal.value_counts(normalize=True)
t2 = df.adverse_loc.value_counts(normalize=True)
t3 = df.adverse_hypoxia.value_counts(normalize=True)
t4 = df.adverse_persistent_hypo.value_counts(normalize=True)
t5 = df.adverse_major_hemorrhage.value_counts(normalize=True)

adverse = pd.concat([t1, t2, t3, t4, t5], axis=1, sort=False)
adverse.columns = ['Failed spinal', 'Loss of consciousness', 'Hypoxia', 'Persistent hypotension', 'Major hemorrhage']
adverse = adverse.transpose()
adverse

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No,Yes
Failed spinal,0.858407,0.141593
Loss of consciousness,0.990868,0.009132
Hypoxia,0.972603,0.027397
Persistent hypotension,0.468750,0.531250
Major hemorrhage,0.990826,0.009174


In [ ]:
#@title
fig = go.Figure(data=[
  go.Bar(name = 'Yes', x=adverse.index, y=adverse['Yes']),
  go.Bar(name = 'No', x=adverse.index, y=adverse['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### What was the response to failed spinal?

In [ ]:
#@title
# replace 'checked' with the response to failed spinal, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'adverse_spinal_failed_y___1':{'Checked':'Convert to GA'}})
df = df.replace({'adverse_spinal_failed_y___2':{'Checked':'Ketamine sedation without intubation'}})

# create dummy variables for indication
t1 = df.adverse_spinal_failed_y___1
t2 = df.adverse_spinal_failed_y___2

#join together dummies
adverse_spinal = pd.concat([t1, t2], axis=0, sort=False)

#remove 'unchecked' values
adverse_spinal = adverse_spinal[adverse_spinal!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Number of failed spinals

In [ ]:
#@title
failed_spinals = df.adverse_failed_spinal.value_counts()
failed_spinals['Yes']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

32

Response to failed spinals (counts)

In [ ]:
#@title
adverse_spinal.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ketamine sedation without intubation    26
Convert to GA                            5
dtype: int64

Response to failed spinals (as percent of failed spinals)

NB currently no Convert to GA. will have to update this if there are some conversions to GA.

In [ ]:
#@title
t1 = adverse_spinal.value_counts()
print('% Ketamine sedation without intubation: ', t1['Ketamine sedation without intubation']/failed_spinals['Yes'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

% Ketamine sedation without intubation:  0.8125


### How much drop in BP?

In [ ]:
#@title
# replace 'checked' the reason for bp treatment, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'adverse_hypo_which___1':{'Checked':'>20% drop'}})
df = df.replace({'adverse_hypo_which___2':{'Checked':'<70mmhg'}})

# create dummy variables for indication
t1 = df.adverse_hypo_which___1
t2 = df.adverse_hypo_which___2

#join together dummies
bpdrop = pd.concat([t1, t2], axis=0, sort=False)

#remove 'unchecked' values
bpdrop = bpdrop[bpdrop!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
bpdrop.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

>20% drop    130
<70mmhg        6
dtype: int64

## End of Case: spinal

### Place of recovery?

In [ ]:
#@title
t1 = df.spinal_end_where.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Corridor                    0.901961
Ward                        0.062092
Designated recovery room    0.035948
Name: spinal_end_where, dtype: float64

In [ ]:
taba, t1 = crossfx('spinal_end_where','phase')
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Phase                    Pre-SAFE           Immediately post SAFE            \
count/pct                       n         %                     n         %   
spinal_end_where                                                              
Corridor                       79  0.840426                    72  0.857143   
Designated recovery room        8  0.085106                     3  0.035714   
Ward                            7  0.074468                     9  0.107143   
All                            94       NaN                    84       NaN   

Phase                    6 months post SAFE           12-mo post SAFE        \
count/pct                                 n         %               n     %   
spinal_end_where                                                              
Corridor                                 87  0.988636              38  0.95   
Designated recovery room                  0  0.000000               0  0.00   
Ward                                      1  0.011364               2  0.05   
All                                      88       NaN              40   NaN   

Phase                     All            
count/pct                   n         %  
spinal_end_where                         
Corridor                  276  0.901961  
Designated recovery room   11  0.035948  
Ward                       19  0.062092  
All                       306       NaN

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Dedicated recovery staff?

In [ ]:
#@title
t1 = df.spinal_end_who.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

No     0.676471
Yes    0.323529
Name: spinal_end_who, dtype: float64

In [ ]:
taba, t1 = crossfx('spinal_end_who', 'phase')
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Phase          Pre-SAFE           Immediately post SAFE            \
count/pct             n         %                     n         %   
spinal_end_who                                                      
No                   80  0.851064                    56  0.666667   
Yes                  14  0.148936                    28  0.333333   
All                  94       NaN                    84       NaN   

Phase          6 months post SAFE           12-mo post SAFE         All  \
count/pct                       n         %               n      %    n   
spinal_end_who                                                            
No                             46  0.522727              25  0.625  207   
Yes                            42  0.477273              15  0.375   99   
All                            88       NaN              40    NaN  306   

Phase                     
count/pct              %  
spinal_end_who            
No              0.676471  
Yes             0.323529  
All                  NaN

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Recovery monitors

In [ ]:
#@title
# replace 'checked' what recovery monitors were used, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'spinal_end_monitors___1':{'Checked':'BP'}})
df = df.replace({'spinal_end_monitors___2':{'Checked':'SpO2'}})
df = df.replace({'spinal_end_monitors___3':{'Checked':'HR'}})

# create dummy variables for indication
t1 = df.spinal_end_monitors___1
t2 = df.spinal_end_monitors___2
t3 = df.spinal_end_monitors___3

#join together dummies
spinal_end_monitors_which = pd.concat([t1, t2, t3], axis=0, sort=False)

#remove 'unchecked' values
spinal_end_monitors_which = spinal_end_monitors_which[spinal_end_monitors_which!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FLAG need again to normalize to total number of cases, this is pure count

In [ ]:
#@title
spinal_end_monitors_which.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BP      47
SpO2    28
HR      28
dtype: int64

In [ ]:
df.pivot_table(index=['phase'], columns=(['spinal_end_monitors___1']), values=['anesthetist_id'], aggfunc=pd.Series.nunique,)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

anesthetist_id          
spinal_end_monitors___1             BP Unchecked
phase                                           
12-mo post SAFE                      4         9
6 months post SAFE                   6        16
Immediately post SAFE                2        21
Pre-SAFE                             4        22

In [ ]:
df.pivot_table(index=['phase'], columns=(['spinal_end_monitors___2']), values=['anesthetist_id'], aggfunc=pd.Series.nunique,)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

anesthetist_id          
spinal_end_monitors___2           SpO2 Unchecked
phase                                           
12-mo post SAFE                      2        10
6 months post SAFE                   3        17
Immediately post SAFE                2        21
Pre-SAFE                             4        22

In [ ]:
df.pivot_table(index=['phase'], columns=(['spinal_end_monitors___3']), values=['anesthetist_id'], aggfunc=pd.Series.nunique,)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

anesthetist_id          
spinal_end_monitors___3             HR Unchecked
phase                                           
12-mo post SAFE                      2        10
6 months post SAFE                   3        17
Immediately post SAFE                2        21
Pre-SAFE                             4        22

### Unstructured spinal observation

In [ ]:
#@title
df.spinal_unstructured.dropna()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0                Surgeon controlled amount of oxytocin. 
1                         Ketamine 50mg + atropine 0.5mg
2                  Patient very restless throughout case
4               Hypotension pre-op which was not managed
5                                ketamine sedation 100mg
                             ...                        
375                            Participant busy on phone
388                          Participant covering 2 ORs.
393        Assistant performed spinal under supervision.
418    Left tilt insufficient.   TXA 1g given  Pain a...
424                                   Wedge insufficient
Name: spinal_unstructured, Length: 166, dtype: object

# General anesthesia packet

## GA Behavior

In [ ]:
#@title
t1 = df.ga_tilt.value_counts(normalize=True)
t2 = df.ga_help.value_counts(normalize=True)
t3 = df.ga_preo2.value_counts(normalize=True)
t4 = df.ga_cricoid.value_counts(normalize=True)
t5 = df.ga_tubeposition.value_counts(normalize=True)
t6 = df.ga_secure.value_counts(normalize=True)
t7 = df.ga_vitals.value_counts(normalize=True)
t8 = df.ga_abx.value_counts(normalize=True)
t9 = df.ga_oxytocin.value_counts(normalize=True)
t10 = df.ga_oxytocintime.value_counts(normalize=True)
t11 = df.ga_present.value_counts(normalize=True)
t12 = df.ga_extubated.value_counts(normalize=True)

ga = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12], axis=1, sort=False)
ga.columns = ['Applies tilt', 'Asks for help', '3min preoxygenation', 'Cricoid pressure', 'Checks tube position', 'Secures ETT', 'Monitors VS', 'Administers IV Abx', 'Administers oxytocin', 'Oxytocin at right time', 'Present in theatre', 'Extubated safely']
ga = ga.transpose()
ga

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No,Yes
Applies tilt,0.611111,0.388889
Asks for help,0.769231,0.230769
3min preoxygenation,0.642857,0.357143
Cricoid pressure,0.833333,0.166667
Checks tube position,0.833333,0.166667
Secures ETT,0.727273,0.272727
Monitors VS,NaN,1.000000
Administers IV Abx,0.111111,0.888889
Administers oxytocin,NaN,1.000000
Oxytocin at right time,0.166667,0.833333


In [ ]:
#@title
fig = go.Figure(data=[
  go.Bar(name = 'Yes', x=ga.index, y=ga['Yes']),
  go.Bar(name = 'No', x=ga.index, y=ga['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### GA: What vital signs were monitored?

In [ ]:
#@title
# replace 'checked' with which vital signs were monitored were used in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'ga_vitals_which___1':{'Checked':'BP every 5 min'}})
df = df.replace({'ga_vitals_which___2':{'Checked':'HR every 5 min'}})
df = df.replace({'ga_vitals_which___3':{'Checked':'Continuous SpO2'}})

# create dummy variables for indication
t1 = df.ga_vitals_which___1
t2 = df.ga_vitals_which___2
t3 = df.ga_vitals_which___3

#join together dummies
ga_vitals = pd.concat([t1, t2, t3], axis=0, sort=False)

#remove 'unchecked' values
ga_vitals = spinal_vitals[spinal_vitals!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

***FLAG*** = need to normalize the following, where the denominator is # of cases

In [ ]:
#@title
t1 = ga_vitals.value_counts()
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BP every 5 min     303
HR every 5 min     302
Continuous SpO2    298
dtype: int64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### GA: Oxytocin dose

In [ ]:
#@title
t1 = df.ga_oxytocin_dose.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

10 units     0.777778
>10 units    0.222222
Name: ga_oxytocin_dose, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### GA: Oxytocin method

In [ ]:
#@title
t1 = df.ga_oxytocin_how.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bolus       0.555556
Both        0.388889
Infusion    0.055556
Name: ga_oxytocin_how, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Extubation

### Extubation components

In [ ]:
#@title
# replace 'checked' with which vital signs were monitored were used in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'ga_extubation_which___1':{'Checked':'Airway suctioned'}})
df = df.replace({'ga_extubation_which___2':{'Checked':'Assess if patient awake'}})
df = df.replace({'ga_extubation_which___3':{'Checked':'Assess for regular breathing'}})

# create dummy variables for indication
t1 = df.ga_extubation_which___1
t2 = df.ga_extubation_which___2
t3 = df.ga_extubation_which___3

#join together dummies
ga_extubation = pd.concat([t1, t2, t3], axis=0, sort=False)

#remove 'unchecked' values
ga_extubation = ga_extubation[ga_extubation!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
t1 = ga_extubation.value_counts()
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Series([], dtype: int64)

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### GA: Adverse events

In [ ]:
#@title
t1 = df.ga_adverse_diffaw.value_counts(normalize=True)
t2 = df.ga_adverse_cicv.value_counts(normalize=True)
t3 = df.ga_adverse_hypoxia.value_counts(normalize=True)
t4 = df.ga_adverse_regurg.value_counts(normalize=True)
t5 = df.ga_adverse_regurg_asp.value_counts(normalize=True)
t6 = df.ga_adverse_hypotension.value_counts(normalize=True)
t7 = df.ga_adverse_bleeding.value_counts(normalize=True)
t8 = df.ga_adverse_seizure.value_counts(normalize=True)
t9 = df.ga_adverse_arrest.value_counts(normalize=True)

ga_adverse = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9], axis=1, sort=False)
ga_adverse.columns = ['Difficult airway', 'Cant intubate, cant ventilate', 'Hypoxia', 'Regurgitation', 'Aspiration', 'Hypotension', 'Bleeding', 'Seizure', 'Arrest']
ga_adverse = ga_adverse.transpose()
ga_adverse

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No,Yes
Difficult airway,1.000000,NaN
"Cant intubate, cant ventilate",1.000000,NaN
Hypoxia,0.666667,0.333333
Regurgitation,0.750000,0.250000
Aspiration,NaN,NaN
Hypotension,1.000000,NaN
Bleeding,1.000000,NaN
Seizure,1.000000,NaN
Arrest,1.000000,NaN


This throws an error because there are no 'yeses'

In [ ]:
#@title
#fig = go.Figure(data=[
#  go.Bar(name = 'Yes', x=ga_adverse.index, y=ga_adverse['Yes']),
#  go.Bar(name = 'No', x=ga_adverse.index, y=ga_adverse['No'])
#])

#fig.update_layout(barmode='group')
#fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Which difficult airway actions were performed?

In [ ]:
#@title
# replace 'checked' with which difficult airway actions in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'ga_diffaw_which___1':{'Checked':'Gets help'}})
df = df.replace({'ga_diffaw_which___2':{'Checked':'Maintains oxygenation'}})
df = df.replace({'ga_diffaw_which___3':{'Checked':'Repositions patient'}})
df = df.replace({'ga_diffaw_which___4':{'Checked':'Applies BURP'}})
df = df.replace({'ga_diffaw_which___5':{'Checked':'Changes laryngoscope blade'}})
df = df.replace({'ga_diffaw_which___6':{'Checked':'Uses bougie or stylet'}})

# create dummy variables for team members
t1 = df.ga_diffaw_which___1
t2 = df.ga_diffaw_which___2
t3 = df.ga_diffaw_which___3
t4 = df.ga_diffaw_which___4
t5 = df.ga_diffaw_which___5
t6 = df.ga_diffaw_which___6

#join together dummies
ga_diffaw = pd.concat([t1, t2, t3, t4, t5, t6], axis=0, sort=False)

#remove 'unchecked' values
ga_diffaw = ga_diffaw[ga_diffaw!='Unchecked']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
t1 = ga_diffaw.value_counts()
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Series([], dtype: int64)

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

When was the hypoxia?

In [ ]:
#@title
# replace 'checked' with which difficult airway actions in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'ga_adverse_hypoxia_when___1':{'Checked':'Gets help'}})
df = df.replace({'ga_adverse_hypoxia_when___2':{'Checked':'Maintains oxygenation'}})
df = df.replace({'ga_adverse_hypoxia_when___3':{'Checked':'Repositions patient'}})
df = df.replace({'ga_adverse_hypoxia_when___4':{'Checked':'Applies BURP'}})

# create dummy variables for team members
t1 = df.ga_adverse_hypoxia_when___1
t2 = df.ga_adverse_hypoxia_when___2
t3 = df.ga_adverse_hypoxia_when___3
t4 = df.ga_adverse_hypoxia_when___4

#join together dummies
ga_hypoxia = pd.concat([t1, t2, t3, t4], axis=0, sort=False)

#remove 'unchecked' values
ga_hypoxia = ga_hypoxia[ga_hypoxia!='Unchecked']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
t1 = ga_hypoxia.value_counts()
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Maintains oxygenation    2
Applies BURP             2
Gets help                1
dtype: int64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Which element of hypotension?

In [ ]:
#@title
# replace 'checked' the reason for bp treatment, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'ga_adverse_hypo_which___1':{'Checked':'>20% drop'}})
df = df.replace({'ga_adverse_hypo_which___2':{'Checked':'<70mmhg'}})

# create dummy variables for indication
t1 = df.ga_adverse_hypo_which___1
t2 = df.ga_adverse_hypo_which___2

#join together dummies
gahypo = pd.concat([t1, t2], axis=0, sort=False)

#remove 'unchecked' values
gahypo = gahypo[gahypo!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
gahypo.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Series([], dtype: int64)

## End of Case: GA

### Place of recovery?

In [ ]:
#@title
t1 = df.ga_end_where.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Corridor    0.777778
Ward        0.222222
Name: ga_end_where, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Dedicated recovery staff?

In [ ]:
#@title
t1 = df.ga_end_who.value_counts(normalize=True)
t1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

No     0.5
Yes    0.5
Name: ga_end_who, dtype: float64

In [ ]:
#@title
fig = go.Figure([go.Bar(x=t1.index, y=t1[0:])])
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Recovery monitors

In [ ]:
#@title
# replace 'checked' what recovery monitors were used, in main dataframe
# note to self: You could do this at the beginning when you open the dataframe
df = df.replace({'ga_end_monitors___1':{'Checked':'BP'}})
df = df.replace({'ga_end_monitors___2':{'Checked':'SpO2'}})
df = df.replace({'ga_end_monitors___3':{'Checked':'HR'}})

# create dummy variables for indication
t1 = df.ga_end_monitors___1
t2 = df.ga_end_monitors___2
t3 = df.ga_end_monitors___3

#join together dummies
ga_end_monitors_which = pd.concat([t1, t2, t3], axis=0, sort=False)

#remove 'unchecked' values
ga_end_monitors_which = ga_end_monitors_which[ga_end_monitors_which!="Unchecked"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FLAG need again to normalize to total number of cases, this is pure count

In [ ]:
#@title
ga_end_monitors_which.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BP      1
SpO2    1
HR      1
dtype: int64

### Unstructured spinal observation

In [ ]:
#@title
df.ga_unstructured.dropna()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

23                    Oxytocin given before cord clamped
55     Haematuria in urine bag. Poor urine output com...
56     Thiopentone 300mg + diazepam 10mg. LMA used to...
106                                    Ketamine sedation
124    Ketamine 150mg given. Additional 150mg given l...
163     - GA without ET done  - Ketamine 100mg IV bol...
177     - Ketamine 100mg IV bolus given (GA without E...
189     - Actions taken during hypoxia, added ketamin...
262     - Failed spinal anaesthesia, ketamine 50mg IV...
263     - 23mins after baby out, mother complaints of...
265     - Failed spinal change to GA  - Ketamine 50mg...
267     - Before fascial cut, patient feels and compl...
268     - 30mins after baby out, mother complaints of...
269     - Left lateral applied but not enough to disp...
275                                 - No intubation done
336          Endobronchial intubation leading to hypoxia
Name: ga_unstructured, dtype: object



---


# Emergency packet

## Peripartum hemorrhage

In [ ]:
#@title
t1 = df.ga_adverse_diffaw.value_counts(normalize=True)
t2 = df.ga_adverse_cicv.value_counts(normalize=True)
t3 = df.ga_adverse_hypoxia.value_counts(normalize=True)
t4 = df.ga_adverse_regurg.value_counts(normalize=True)
t5 = df.ga_adverse_regurg_asp.value_counts(normalize=True)
t6 = df.ga_adverse_hypotension.value_counts(normalize=True)
t7 = df.ga_adverse_bleeding.value_counts(normalize=True)
t8 = df.ga_adverse_seizure.value_counts(normalize=True)
t9 = df.ga_adverse_arrest.value_counts(normalize=True)

ga_adverse = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9], axis=1, sort=False)
ga_adverse.columns = ['Difficult airway', 'Cant intubate, cant ventilate', 'Hypoxia', 'Regurgitation', 'Aspiration', 'Hypotension', 'Bleeding', 'Seizure', 'Arrest']
ga_adverse = ga_adverse.transpose()
ga_adverse

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No,Yes
Difficult airway,1.000000,NaN
"Cant intubate, cant ventilate",1.000000,NaN
Hypoxia,0.666667,0.333333
Regurgitation,0.750000,0.250000
Aspiration,NaN,NaN
Hypotension,1.000000,NaN
Bleeding,1.000000,NaN
Seizure,1.000000,NaN
Arrest,1.000000,NaN


In [ ]:
#@title
fig = go.Figure(data=[
  go.Bar(name = 'Yes', x=ga_adverse.index, y=ga_adverse['Yes']),
  go.Bar(name = 'No', x=ga_adverse.index, y=ga_adverse['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Approach to collapsed/unconscious patient

There appear to be no values filled out here.

In [ ]:
#@title
t1 = df.adverse_arrest_help.value_counts(normalize=True)
t2 = df.adverse_arrest_airway.value_counts(normalize=True)
t3 = df.adverse_arrest_breathing.value_counts(normalize=True)
t4 = df.adverse_arrest_o2.value_counts(normalize=True)
t5 = df.adverse_arrest_pulse.value_counts(normalize=True)
t6 = df.adverse_arrest_tilt.value_counts(normalize=True)
t7 = df.adverse_arrest_fluids.value_counts(normalize=True)
t8 = df.adverse_arrest_bp.value_counts(normalize=True)
t9 = df.adverse_arrest_communicate.value_counts(normalize=True)

adverse_arrest = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9], axis=1, sort=False)
adverse_arrest.columns = ['Asks for help', 'Assesses/opens airway', 'Assesses breathing/SpO2', 'High flow o2', 'Checks pulse', 'Applies/checks tilt', 'Checks IV fluids/running', 'Checks BP', 'Communicates with surgeon/OB']
adverse_arrest = adverse_arrest.transpose()
adverse_arrest

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

""
Asks for help
Assesses/opens airway
Assesses breathing/SpO2
High flow o2
Checks pulse
Applies/checks tilt
Checks IV fluids/running
Checks BP
Communicates with surgeon/OB


## Neonatal resuscitation

In [ ]:
#@title
t1 = df.adverse_neo_attempt.value_counts(normalize=True)
t2 = df.adverse_neo_timer.value_counts(normalize=True)
t3 = df.adverse_neo_dries.value_counts(normalize=True)
t4 = df.adverse_neo_towel.value_counts(normalize=True)
t5 = df.adverse_neo_airway.value_counts(normalize=True)
t6 = df.adverse_neo_breathing.value_counts(normalize=True)
t7 = df.adverse_neo_hr.value_counts(normalize=True)
t8 = df.adverse_neo_5breath.value_counts(normalize=True)
t9 = df.adverse_neo_5morebreath.value_counts(normalize=True)
t10 = df.adverse_neo_compressions.value_counts(normalize=True)

adverse_neo = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9, t10], axis=1, sort=False)
adverse_neo.columns = ['Attempts to resuscitate', 'Starts timer', 'Dries baby', 'Keeps warm + towel', 'Opens airway', 'Assesses breathing', 'Assesses heart beat', 'Gives 5 breaths', 'Gives 5 more breaths', 'Performs CPR for HR<100']
adverse_neo = adverse_neo.transpose()
adverse_neo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Yes,No
Attempts to resuscitate,1.000000,NaN
Starts timer,NaN,1.000000
Dries baby,0.900000,0.100000
Keeps warm + towel,0.800000,0.200000
Opens airway,0.900000,0.100000
Assesses breathing,1.000000,NaN
Assesses heart beat,0.400000,0.600000
Gives 5 breaths,0.800000,0.200000
Gives 5 more breaths,0.714286,0.285714
Performs CPR for HR<100,0.500000,0.500000


In [ ]:
#@title
t1 = df.adverse_neo_attempt.value_counts(normalize=False)
t2 = df.adverse_neo_timer.value_counts(normalize=False)
t3 = df.adverse_neo_dries.value_counts(normalize=False)
t4 = df.adverse_neo_towel.value_counts(normalize=False)
t5 = df.adverse_neo_airway.value_counts(normalize=False)
t6 = df.adverse_neo_breathing.value_counts(normalize=False)
t7 = df.adverse_neo_hr.value_counts(normalize=False)
t8 = df.adverse_neo_5breath.value_counts(normalize=False)
t9 = df.adverse_neo_5morebreath.value_counts(normalize=False)
t10 = df.adverse_neo_compressions.value_counts(normalize=False)

adverse_neo = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9, t10], axis=1, sort=False)
adverse_neo.columns = ['Attempts to resuscitate', 'Starts timer', 'Dries baby', 'Keeps warm + towel', 'Opens airway', 'Assesses breathing', 'Assesses heart beat', 'Gives 5 breaths', 'Gives 5 more breaths', 'Performs CPR for HR<100']
adverse_neo = adverse_neo.transpose()
adverse_neo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Yes,No
Attempts to resuscitate,10.0,NaN
Starts timer,NaN,10.0
Dries baby,9.0,1.0
Keeps warm + towel,8.0,2.0
Opens airway,9.0,1.0
Assesses breathing,10.0,NaN
Assesses heart beat,4.0,6.0
Gives 5 breaths,8.0,2.0
Gives 5 more breaths,5.0,2.0
Performs CPR for HR<100,2.0,2.0


In [ ]:
#@title
fig = go.Figure(data=[
  go.Bar(name = 'Yes', x=adverse_neo.index, y=adverse_neo['Yes']),
  go.Bar(name = 'No', x=adverse_neo.index, y=adverse_neo['No'])
])

fig.update_layout(barmode='group')
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Other adverse events

In [ ]:
#@title
t1 = df.adverse_other_equip.value_counts(normalize=True)
t2 = df.adverse_other_power.value_counts(normalize=True)
t3 = df.adverse_other_o2.value_counts(normalize=True)

adverse_other = pd.concat([t1, t2, t3], axis=1, sort=False)
adverse_other.columns = ['Anesthesia equipment failure', 'Power failure', 'Loss of oxygen']
adverse_other = adverse_other.transpose()
adverse_other

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

""
Anesthesia equipment failure
Power failure
Loss of oxygen


In [ ]:
#@title
df.adverse_unstructured.dropna()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

13     Heater not turned on, although baby covered. S...
25     Significant intra-op blood loss. Probably >2L....
54     Baby responded after stimulation and airway op...
189                           - Resuscitation successful
244                               - Baby did not survive
261     - Soon after placenta delivery, bleeding star...
273                            Successful resuscitation 
275     - Resuscitation successful  - The neonate req...
Name: adverse_unstructured, dtype: object

# Post-Safe 1 comparision

# Table 1

In [ ]:
taba, t1 = crossfx('hospital', 'phase')
t1=t1.reset_index()
t1 = t1.set_index('hospital')
t1['tabletype'] = 'hospital'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
taba, t2 = crossfx('cadre', 'phase')
t2 = t2.reset_index()
t2 = t2.set_index('cadre')
t2['tabletype'] = 'cadre'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
taba, t3 = crossfx('case_urgency', 'phase')
t3 = t3.reset_index().set_index('case_urgency')
t3['tabletype'] = 'case_urgency'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
t4 = indicationsum.reset_index().set_index('index')
t4['tabletype'] = 'spinal_indication'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Simply remove the group by if you need to access each row

In [ ]:
ta = pd.concat([t1,t2,t3,t4], axis=0)
ta
ta.groupby(['tabletype', ta.index]).mean()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



Pre-SAFE            \
                                                      n         %   
tabletype                                                           
cadre             All                              99.0       NaN   
                  Assistant nurse anaesthetist     15.0  0.151515   
                  Nurse anesthetist                84.0  0.848485   
                  Other (specify)                   0.0  0.000000   
case_urgency      All                              98.0       NaN   
                  Elective                         25.0  0.255102   
                  Emergency                        73.0  0.744898   
hospital          All                             100.0       NaN   
                  IR                               16.0  0.160000   
                  MB                               30.0  0.300000   
                  NJ                                4.0  0.040000   
                  R                                18.0  0.180000   
                  Z                                32.0  0.320000   
spinal_indication Breech presentation               0.0  0.000000   
                  Eclampsia                         0.0  0.000000   
                  Fetal distress                   11.0  0.110000   
                  Multiple pregnancy                1.0  0.010000   
                  Obstructed labour                22.0  0.220000   
                  Other                            16.0  0.160000   
                  Placental abruption               1.0  0.010000   
                  Severe pre-eclampsia              0.0  0.000000   
                  previous scars                   51.0  0.510000   

                                               Immediately post SAFE  \
                                                                   n   
tabletype                                                              
cadre             All                                           89.0   
                  Assistant nurse anaesthetist                  14.0   
                  Nurse anesthetist                             75.0   
                  Other (specify)                                0.0   
case_urgency      All                                           89.0   
                  Elective                                      28.0   
                  Emergency                                     61.0   
hospital          All                                           88.0   
                  IR                                             7.0   
                  MB                                            19.0   
                  NJ                                            11.0   
                  R                                             12.0   
                  Z                                             39.0   
spinal_indication Breech presentation                            4.0   
                  Eclampsia                                      1.0   
                  Fetal distress                                 5.0   
                  Multiple pregnancy                             5.0   
                  Obstructed labour                             16.0   
                  Other                                         16.0   
                  Placental abruption                            0.0   
                  Severe pre-eclampsia                           4.0   
                  previous scars                                46.0   

                                                         6 months post SAFE  \
                                                       %                  n   
tabletype                                                                     
cadre             All                                NaN               88.0   
                  Assistant nurse anaesthetist  0.157303                1.0   
                  Nurse anesthetist             0.842697               82.0   
                  Other (specify)             

# Table 2

In [ ]:
from IPython.display import display, HTML
from jinja2 import Template

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# import stats
df2 = pd.read_csv('https://raw.githubusercontent.com/lawtj/safe/master/safestats.csv')
#df2 = pd.read_csv('/content/drive/MyDrive/Docs to edit and return by email/output.txt', sep='\t+')

# import variable labels
desc = pd.read_csv('https://raw.githubusercontent.com/lawtj/safe/master/vardesc.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

df2 = df2.drop(['label'], axis=1)
df2 = df2.merge(desc, on='variable', how='left')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
spinalvars = ['spinal_hat',
 'spinal_mask',
 'spinal_gloves',
 'spinal_clean',
 'spinal_sterile',
 'spinal_wedge',
 'spinal_vitals',
 'spinal_height',
 'spinal_vasopressor',
 'spinal_abx',
 'spinal_oxytocin',
 'spinal_oxytocintime',
 'spinal_present',
 'adverse_persistent_hypo']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
alltab = pd.DataFrame()

for j in [commvars,preplist,spinalvars]:
    for i in j:
        t1 = crossfx2(i,'phase')
        if 'No' in t1.index:
            t1=t1.drop(['No'])
        else:
            continue
        t1=t1.drop(['All'])
        t1.index=[i]
        t1['tabletype'] = i
        alltab = pd.concat([alltab, t1],axis=0)

def tablety(row):
    if row['tabletype'] in commvars:
        return'commvars'
    elif row['tabletype'] in preplist:
        return 'preplist'
    elif row['tabletype'] in spinalvars:
        return 'spinalvars'

alltab['tabletype'] = alltab.apply(tablety, axis=1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
t2a = alltab.merge(df2, left_index=True, right_on='variable', how='outer')
t2a = t2a.set_index('variable')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Format p values

In [ ]:
# deal with p values of nan, <.001, or greater.

def pval(pval):
    if pd.isnull(pval):
        return '-'
    elif pval <= 0.001:
        return '<.001'
    else: 
        return round(pval,3)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## N, % as separate

In [ ]:
phaselist = ['Pre-SAFE','Immediately post SAFE','6 months post SAFE','12-mo post SAFE','All']


table_str='''

<html>
<head>
<style>
.table-style {
    border: 1px solid white;
    text-align: center;
    border-collapse: collapse;
}

.table-style td{
    padding: 5px 5px;
}

.darkgray {
    background-color: #d9d9d9;
}

.section-break {
    border-top: 1.5pt solid black;
    border-bottom: 1.5pt solid black;
    background-color: #efefef;
}

.bottom-solid-hard{
    border-bottom: 1.5pt solid black;
}

.left-solid {
    border-left: 1pt solid black;
}

.right-solid {
    border-right: 1pt solid black;
}

.right-dash {
    border-right: 1pt dotted black;
}


</style>
</head>

<table class='table-style'>
    <thead>
    <tr class='darkgray'>
        <th></th>
    
    {# ######## headers #########          #}
        {# ######## top header #########          #}

        {% for i in phaselist %}
            <th colspan=2 class='left-solid right-solid'>{{i}} (N={{df['phase'].value_counts().loc[i]}}) </th>
        {% endfor %}
    
        <th></th>
    </tr>
    
    <tr class='darkgray bottom-solid-hard'>
        <th></th>
        {# ######## n, % headers #########          #}

        {% for i in phaselist %}
            <th class='left-solid'>{{'n'}}</th>
            <th class='right-solid'>{{'%'}}</th>
        {% endfor %}
    
        <th>Chi2</th>
    </tr>
    </thead>
    
    <tbody>

{% for varlist, sec in zip(['commvars','preplist','spinalvars'], ['Communication and WHO Checklist', 'Peroperative preparation','Spinal management'])  %}
    <tr class='section-break'> <td colspan=12>{{sec}}</td></tr>

    {% for i,x in zip(t2a[t2a['tabletype'] == varlist].index, t2a[t2a['tabletype'] == varlist]['label']) %}
        <tr>
        <th>{{x}}</th>
        {% for j in phaselist %}
            <td class='left-solid right-dash'>{{t2a.at[i, j~'_n']|int}} </td>
            <td class='right-solid'>{{(t2a.at[i, j~'_%']*100)|int}} </td>
        {% endfor %}
            {% if pd.notnull(t2a.at[i,'p_value']) %}
                <td>{{t2a.at[i,'p_value']|round(3)}}</td>
            {% elif pd.isnull(t2a.at[i,'exact']) %}
                <td>-</td>
            {% else %}
                <td>{{t2a.at[i,'exact']|round(3)}}*</td>
            {% endif %}
        </tr>
    {%endfor%}
{% endfor %}

    <tr>
        <td>* = Fischer's exact
    </tr>
    
    </tbody>
</table>

</html>

'''

template=Template(table_str)
HTML(template.render(phaselist = phaselist,df=df, commnames=commnames, prepnames=prepnames, t2a=t2a, zip=zip, pd=pd))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(template.render(phaselist = phaselist,df=df, commnames=commnames, prepnames=prepnames, t2a=t2a, zip=zip, pd=pd, pval=pval))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



<html>
<head>
<style>
.table-style {
    border: 1px solid white;
    text-align: center;
    border-collapse: collapse;
}

.table-style td{
    padding: 5px 5px;
}

.darkgray {
    background-color: #d9d9d9;
}

.section-break {
    border-top: 1.5pt solid black;
    border-bottom: 1.5pt solid black;
    background-color: #efefef;
}

.bottom-solid-hard{
    border-bottom: 1.5pt solid black;
}

.left-solid {
    border-left: 1pt solid black;
}

.right-solid {
    border-right: 1pt solid black;
}

.right-dash {
    border-right: 1pt dotted black;
}


</style>
</head>

<table class='table-style'>
    <thead>
    <tr class='darkgray'>
        <th></th>
    
    
        

        
            <th colspan=2 class='left-solid right-solid'>Pre-SAFE (N=100) </th>
        
            <th colspan=2 class='left-solid right-solid'>Immediately post SAFE (N=89) </th>
        
            <th colspan=2 class='left-solid right-solid'>6 months post SAFE (N=89) </th>
        
            <th colspan=

## N % in one

In [ ]:
phaselist = ['Pre-SAFE','Immediately post SAFE','6 months post SAFE','12-mo post SAFE','All']


table_str='''

<html>
<head>
<style>
.table-style {
    border: 1px solid white;
    text-align: center;
    border-collapse: collapse;
}

.darkgray {
    background-color: #d9d9d9;
}

.section-break {
    border-top: 1.5pt solid black;
    border-bottom: 1.5pt solid black;
    background-color: #efefef;
}

.bottom-solid-hard{
    border-bottom: 1.5pt solid black;
}

.left-solid {
    border-left: 1pt solid black;
}

.right-solid {
    border-right: 1pt solid black;
}

.right-dash {
    border-right: 1pt dotted black;
}


</style>
</head>

<table class='table-style'>
    <thead>
    <tr class='darkgray'>
        <th></th>
    
    {# ######## headers #########          #}
        {# ######## top header #########          #}

        {% for i in phaselist %}
            <th class='left-solid right-solid'>{{i}} (N={{df['phase'].value_counts().loc[i]}}) </th>
        {% endfor %}
    
        <th></th>
    </tr>
    
    <tr class='darkgray bottom-solid-hard'>
        <th></th>
        {# ######## n, % headers #########          #}

        {% for i in phaselist %}
            <th class='left-solid'>n (%)</th>
        {% endfor %}
    
        <th>Chi2</th>
    </tr>
    </thead>
    
    <tbody>


    
{% for varlist, sec in zip(['commvars','preplist','spinalvars'], ['Communication and WHO Checklist', 'Peroperative preparation','Spinal management'])  %}
    <tr class='section-break'> <td colspan=7> {{sec}} </td></tr>

    {% for i,x in zip(t2a[t2a['tabletype'] == varlist].index, t2a[t2a['tabletype'] == varlist]['label']) %}
        <tr>
        <th>{{x}}</th>
        {% for j in phaselist %}
            <td class='left-solid right-dash'>{{t2a.at[i, j~'_n']|int}} ({{(t2a.at[i, j~'_%']*100)|round|int}}) </td>
        {% endfor %}
            {% if pd.notnull(t2a.at[i,'p_value']) %}
                <td>{{t2a.at[i,'p_value']|round(3)}}</td>
            {% elif pd.isnull(t2a.at[i,'exact']) %}
                <td>-</td>
            {% else %}
                <td>{{t2a.at[i,'exact']|round(3)}}*</td>
            {% endif %}
        </tr>
    {%endfor%}
{% endfor %}

    <tr>
        <td>* = Fischer's exact
    </tr>
    
    </tbody>
</table>

</html>

'''

template=Template(table_str)
HTML(template.render(phaselist = phaselist,df=df, commnames=commnames, prepnames=prepnames, t2a=t2a, pval=pval, zip=zip, pd=pd))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(template.render(phaselist = phaselist,df=df, commnames=commnames, prepnames=prepnames, t2a=t2a, zip=zip, pd=pd, pval=pval))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



<html>
<head>
<style>
.table-style {
    border: 1px solid white;
    text-align: center;
    border-collapse: collapse;
}

.darkgray {
    background-color: #d9d9d9;
}

.section-break {
    border-top: 1.5pt solid black;
    border-bottom: 1.5pt solid black;
    background-color: #efefef;
}

.bottom-solid-hard{
    border-bottom: 1.5pt solid black;
}

.left-solid {
    border-left: 1pt solid black;
}

.right-solid {
    border-right: 1pt solid black;
}

.right-dash {
    border-right: 1pt dotted black;
}


</style>
</head>

<table class='table-style'>
    <thead>
    <tr class='darkgray'>
        <th></th>
    
    
        

        
            <th class='left-solid right-solid'>Pre-SAFE (N=100) </th>
        
            <th class='left-solid right-solid'>Immediately post SAFE (N=89) </th>
        
            <th class='left-solid right-solid'>6 months post SAFE (N=89) </th>
        
            <th class='left-solid right-solid'>12-mo post SAFE (N=43) </th>
        
           

# Table 3

In [ ]:
orlist = ['or_immediate', 'lci_imm', 'uci_imm', 'p_imm']
t2p = t2a[((t2a['tabletype'] == 'preplist') & ((t2a['p_value'] <=.05) | (t2a['exact'] <=.05)) )]
t2s = t2a[((t2a['tabletype'] == 'spinalvars') & ((t2a['p_value'] <=.05) | (t2a['exact'] <=.05)) )]

table_str='''

<html>

<head>
<style>
.table-style {
    border: 1px solid white;
    text-align: center;
    border-collapse: collapse;
}

.table-style td{
    padding: 5px 5px;
}

.darkgray {
    background-color: #d9d9d9;
}

.section-break {
    border-top: 1.5pt solid black;
    border-bottom: 1.5pt solid black;
    background-color: #efefef;
}

.top-solid-hard{
    border-top: 1.5pt solid black;
}

.bottom-solid-hard{
    border-bottom: 1.5pt solid black;
}

.left-solid {
    border-left: 1pt solid black;
}

.right-solid {
    border-right: 1pt solid black;
}

.right-dash {
    border-right: 1pt dotted black;
}


</style>
</head>

<table class='table-style'>
    <thead>
        <tr class='darkgray'>
            <th></th>
            {% for i in ['Immediately post SAFE', '6 months post SAFE', '12-mo post SAFE'] %}
                <th colspan=2 class='left-solid'>{{i}}</th>
            {% endfor %}
        </tr>

        <tr class='darkgray'>
            <th></th>
            {% for i in range(3) %}
                <th class='left-solid'> OR (95% CI) </th>
                <th class='right-solid'> p </th>
            {% endfor %}

        <tr class='header darkgray top-solid-hard'> <td colspan=7> Unadjusted analysis </td></tr>

    </thead>

    
    <tbody>

{# ######## communicaton variables #########          #}
        <tr class='section-break'> <td colspan=7> Communication </td></tr>
        {% for i,x in zip(t2a[t2a['tabletype'] == 'commvars'].index, t2a['label']) %}
            <tr>
                <th>{{x}}</th>
                <td class='left-solid right-dash'>{{t2a.at[i, 'or_immediate']|round(2)}} ({{t2a.at[i, 'lci_imm']|round(2)}} - {{t2a.at[i, 'uci_imm']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p_imm'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'OR6']|round(2)}} ({{t2a.at[i, 'lci6']|round(2)}} - {{t2a.at[i, 'uci6']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p_6'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'OR12']|round(2)}} ({{t2a.at[i, 'lci12']|round(2)}} - {{t2a.at[i, 'uci12']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p12'])}} </td>
            </tr>
        {%endfor%}

{# ######## prep variables #########          #}
        <tr class='section-break'> <td colspan=7> Preop Checks </td></tr>
        {% for i,x in zip(t2p.index, t2p['label']) %}
            <tr>
                <th>{{x}}</th>
                <td class='left-solid right-dash'>{{t2a.at[i, 'or_immediate']|round(2)}} ({{t2a.at[i, 'lci_imm']|round(2)}} - {{t2a.at[i, 'uci_imm']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p_imm'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'OR6']|round(2)}} ({{t2a.at[i, 'lci6']|round(2)}} - {{t2a.at[i, 'uci6']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p_6'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'OR12']|round(2)}} ({{t2a.at[i, 'lci12']|round(2)}} - {{t2a.at[i, 'uci12']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p12'])}} </td>
            </tr>
        {% endfor %}

{# ######## spinal variables #########          #}
        <tr class='section-break'> <td colspan=7> Spinal variables </td></tr>
        {% for i,x in zip(t2s.index, t2s['label']) %}
            <tr>
                <th>{{x}}</th>
                <td class='left-solid right-dash'>{{t2a.at[i, 'or_immediate']|round(2)}} ({{t2a.at[i, 'lci_imm']|round(2)}} - {{t2a.at[i, 'uci_imm']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p_imm'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'OR6']|round(2)}} ({{t2a.at[i, 'lci6']|round(2)}} - {{t2a.at[i, 'uci6']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p_6'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'OR12']|round(2)}} ({{t2a.at[i, 'lci12']|round(2)}} - {{t2a.at[i, 'uci12']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'p12'])}} </td>
            </tr>
        {% endfor %}

{# ######## ADJUSTED VARIABLES #########          #}
{# ######## communicaton adjusted variables #########          #}
        <tr class='darkgray top-solid-hard'> <td colspan=7> Adjusted analysis </td></tr>

        <tr class='section-break'> <td colspan=7> Communication adjusted </td></tr>
        {% for i,x in zip(t2a[t2a['tabletype'] == 'commvars'].index, t2a['label']) %}
            <tr>
                <th>{{x}}</th>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_or_immediate']|round(2)}} ({{t2a.at[i, 'adj_lci_imm']|round(2)}} - {{t2a.at[i, 'adj_uci_imm']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p_imm'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_OR6']|round(2)}} ({{t2a.at[i, 'adj_lci6']|round(2)}} - {{t2a.at[i, 'adj_uci6']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p_6'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_OR12']|round(2)}} ({{t2a.at[i, 'adj_lci12']|round(2)}} - {{t2a.at[i, 'adj_uci12']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p12'])}} </td>
            </tr>
        {%endfor%}

{# ######## prep adjusted variables #########          #}
        <tr class='section-break'> <td colspan=7> Preop adjusted Checks </td></tr>
        {% for i,x in zip(t2p.index, t2p['label']) %}
            <tr>
                <th>{{x}}</th>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_or_immediate']|round(2)}} ({{t2a.at[i, 'adj_lci_imm']|round(2)}} - {{t2a.at[i, 'adj_uci_imm']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p_imm'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_OR6']|round(2)}} ({{t2a.at[i, 'adj_lci6']|round(2)}} - {{t2a.at[i, 'adj_uci6']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p_6'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_OR12']|round(2)}} ({{t2a.at[i, 'adj_lci12']|round(2)}} - {{t2a.at[i, 'adj_uci12']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p12'])}} </td>
            </tr>
        {% endfor %}

{# ######## spinal variables #########          #}
        <tr class='section-break'> <td colspan=7> Spinal adjusted variables </td></tr>
        {% for i,x in zip(t2s.index, t2s['label']) %}
            <tr>
                <th>{{x}}</th>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_or_immediate']|round(2)}} ({{t2a.at[i, 'adj_lci_imm']|round(2)}} - {{t2a.at[i, 'adj_uci_imm']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p_imm'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_OR6']|round(2)}} ({{t2a.at[i, 'adj_lci6']|round(2)}} - {{t2a.at[i, 'adj_uci6']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p_6'])}} </td>
                <td class='left-solid right-dash'>{{t2a.at[i, 'adj_OR12']|round(2)}} ({{t2a.at[i, 'adj_lci12']|round(2)}} - {{t2a.at[i, 'adj_uci12']|round(2)}})</td>
                <td>{{pval(t2a.at[i,'adj_p12'])}} </td>
            </tr>
        {% endfor %}

    </tbody>
</table>

</html>

'''

template=Template(table_str)
HTML(template.render(phaselist=phaselist, orlist=orlist, t2a=t2a, t2p=t2p, t2s=t2s, zip=zip, pval=pval, commnames=commnames, prepnames=prepnames, spinalvars=spinalvars, rdp=3))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(template.render(phaselist=phaselist, orlist=orlist, t2a=t2a, t2p=t2p, t2s=t2s, zip=zip, pval=pval, commnames=commnames, prepnames=prepnames, spinalvars=spinalvars, rdp=3))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



<html>

<head>
<style>
.table-style {
    border: 1px solid white;
    text-align: center;
    border-collapse: collapse;
}

.table-style td{
    padding: 5px 5px;
}

.darkgray {
    background-color: #d9d9d9;
}

.section-break {
    border-top: 1.5pt solid black;
    border-bottom: 1.5pt solid black;
    background-color: #efefef;
}

.top-solid-hard{
    border-top: 1.5pt solid black;
}

.bottom-solid-hard{
    border-bottom: 1.5pt solid black;
}

.left-solid {
    border-left: 1pt solid black;
}

.right-solid {
    border-right: 1pt solid black;
}

.right-dash {
    border-right: 1pt dotted black;
}


</style>
</head>

<table class='table-style'>
    <thead>
        <tr class='darkgray'>
            <th></th>
            
                <th colspan=2 class='left-solid'>Immediately post SAFE</th>
            
                <th colspan=2 class='left-solid'>6 months post SAFE</th>
            
                <th colspan=2 class='left-solid'>12-mo post SAFE</th>
            
      

# Figure 1 - number of participant observations per phase

In [ ]:
t1 = pd.crosstab(df['anesthetist_id'], df['phase'])
px.imshow(t1.T, color_continuous_scale='gray', text_auto=True, labels=dict(x="Participant", y="Phase", color="# of observations"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
px.imshow(t1.T, color_continuous_scale='YlGn', text_auto=True, labels=dict(x="Participant", y="Phase", color="# of observations"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>